In [1]:
# 04_model_evaluation.ipynb

# Step 1: Setup & Imports
import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from tensorflow.keras.models import load_model

# Add src path
sys.path.append("..")

from src.model_config import ModelConfig
from src.data_preprocessing import DataLoader
from src.utils import (
    setup_logging, print_system_info,
    calculate_metrics, plot_confusion_matrix,
    save_metrics
)

# Step 2: Load Configuration
config = ModelConfig()
data_params = config.get_data_params()
evaluation_config = config.evaluation_config
paths_config = config.paths_config

# Setup logging and system info
setup_logging()
print_system_info()

# Step 3: Load Test Dataset
loader = DataLoader(
    dataset_path=data_params["dataset_path"],
    processed_path=data_params["processed_path"],
    image_size=tuple(data_params["image_size"]),
    seed=data_params["random_seed"]
)

_, _, test_ds, class_names = loader.get_tf_datasets(
    batch_size=data_params["batch_size"]
)

# Step 4: Load Best Model
checkpoint_path = Path(paths_config["checkpoint_path"]) / "best_model.h5"
model = load_model(checkpoint_path)

# Step 5: Predict on Test Set
y_true = []
y_pred = []
y_pred_proba = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_pred_proba.extend(preds)
    
    if preds.shape[1] == 1:
        # Binary classification sigmoid
        preds_labels = (preds > evaluation_config['threshold']).astype(int)
    else:
        # Multiclass softmax
        preds_labels = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds_labels)

# Step 6: Compute Metrics
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

metrics = calculate_metrics(
    y_true, y_pred,
    y_pred_proba if y_pred_proba.shape[1] > 1 else None
)

# Step 7: Save Metrics and Confusion Matrix
experiment_dir = Path("results/experiments") / sorted(os.listdir("results/experiments"))[-1]
metrics_path = experiment_dir / "evaluation_metrics.json"
cm_path = experiment_dir / "confusion_matrix.png"

save_metrics(metrics, str(metrics_path))
plot_confusion_matrix(y_true, y_pred, class_names=class_names, save_path=str(cm_path))

# Step 8: Display Results
from pprint import pprint
pprint(metrics)

print(f"\n📈 Metrics saved at: {metrics_path}")
print(f"📊 Confusion matrix saved at: {cm_path}")

FileNotFoundError: Configuration file not found: c:\Users\HP\OneDrive\Desktop\leukemia_detection\notebooks\..\src\config.yaml